In [1]:
import torch
import torchtext
import numpy as np
import torchtext
import pandas
import collections
import collections
from torchnlp import *
from torch.optim import lr_scheduler

# Обучение Simple RNN с посимвольной и по-словной токенизацией

### гиперпараметры

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# токенизацией по словам
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
# длина обрабатываемой последовательности токенов
nchars = 100
num_layers=3
test_str_orig = "Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли у него по бокам. Лорд Малфой повернул голову, слишком слабо, чтобы считать это знаком внимания, но всё-таки в направлении профессора Защиты."
test_str_slov = "Беловолосый человек остановился у самого"
test_str = test_str_orig[:nchars]


## Загрузка датасета с токенизацией по словам

In [3]:
def load_dataset_HP(ngrams=1,min_freq=1):
    global vocab, tokenizer
    print("Loading dataset...")
    train_dataset = pandas.read_csv('./dataHPMRM.csv')["text"]
    train_dataset = list(train_dataset)
    print('Building vocab...')
    counter = collections.Counter()
    for line in train_dataset:
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
    vocab = torchtext.vocab.vocab(counter, min_freq=min_freq)
    return train_dataset,vocab
train_dataset,vocab_slov=load_dataset_HP()
vocab_size_slov = len(vocab_slov)

Loading dataset...
Building vocab...


### Посимвольная токенизация

In [4]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for line in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")


Vocabulary size = 167
Encoding of 'a' is 120
Character with code 13 is н


### Функции Кодирования токенов

In [5]:
stoi_hash = {}
def encode(x,voc=None,unk=0,tokenizer=tokenizer):
    global stoi_hash
    v = vocab if voc is None else voc
    if v in stoi_hash.keys():
        stoi = stoi_hash[v]
    else:
        stoi = v.get_stoi()
        stoi_hash[v]=stoi        
    return [stoi.get(s,unk) for s in tokenizer(x)]

In [6]:
## Ф-ии Кодирование токинов по словам
def enc_slov(x):
    return torch.LongTensor(encode(x,voc=vocab_slov,tokenizer=tokenizer))

In [7]:
## Ф-ии Кодирование токинов по символам
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

### Функция Создания Батча в эпохе

In [8]:
def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

In [9]:
def get_batch_slov(s,nchars=nchars):
    leng = len(enc_slov(s))
    inps = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    for i in range(leng-nchars):
        inps[i] = enc_slov(s)[i:i+nchars]
        outs[i] = enc_slov(s)[i+1:i+nchars+1]
    return inps,outs

## Однослойная однонаправленая сеть Simple RNN

In [10]:
class RNNGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size=vocab_size
        self.rnn = torch.nn.RNN(self.vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, self.vocab_size)

    def forward(self, x):
        x = torch.nn.functional.one_hot(x,self.vocab_size).to(torch.float32)
        x,h = self.rnn(x)
        return self.fc(x)

## Генератор следующией послдовательности символов/слов

In [11]:
def generate(net,size=100,start='Сегодня '):
    chars = list(start)
    out = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars.append(vocab.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)
        
def generate_slov(net,size=100,start='Сегодня '):
    chars = start
    out = net(enc_slov(start).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars+= " " + (vocab_slov.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)

## Мягкая генерация

In [25]:
def generate_soft(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = list(start)
    out = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars.append(vocab.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)

def generate_soft_slov(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = start
    out = net(enc_slov(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars += " " + vocab_slov.get_itos()[nc]
        out = net(nc.view(1,-1))
    return ''.join(chars)


## Ф-ия обучения

In [13]:
def train(net,train_dataset,val_dataset,num_epochs, optimizer, scheduler, get_batch, generate, slov = False):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for phase in ["T","V"]:
            if phase == 'T':
                net.train()  # Установить модель в режим обучения
                dataset=train_dataset
            else:
                net.eval()   #Установить модель в режим оценки
                dataset=val_dataset
            
            epoch_loss = 0.0    
            for i,x in enumerate(dataset):
                
                # x[0] is class label, x is text
                if slov:
                    leng = len(enc_slov(x))
                else:
                    leng = len(x)
                    
                if leng-nchars<10:
                    continue

                text_in, text_out = get_batch(x,nchars=nchars)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'T'):
                    out = net(text_in)
                    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
                    if phase == 'T':
                        loss.backward()
                        optimizer.step()
                    else:
                        # scheduler.step(loss.item())
                        pass
                epoch_loss+=loss.item()
                # if i%100==0:
                #     print(f"{i}/{len(dataset)} Current loss {phase} = {epoch_loss/(i+1)}")
            print(f"Current loss {phase} = {epoch_loss / len(dataset)}")
            if phase == 'V':
                print(generate(net,start=test_str))    

## Обучение Simple RNN с посимвольной токенизацией

In [14]:
net=RNNGenerator(vocab_size,nchars).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [15]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch, generate)

Epoch 0/10
Current loss T = 1.0830420300127306
Current loss V = 1.028870946333628
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли стори стори стори стори стори стори стори стори стори стори стори стори стори стори стори стори ст
Epoch 1/10
Current loss T = 1.1287643617113217
Current loss V = 1.2247824693697826
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто ст
Epoch 2/10
Current loss T = 1.1782178978455407
Current loss V = 1.165402637884211
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто сто ст
Epoch 3/10
Current loss T = 1.1574416881218978
Current loss V = 1.1554023067142574
Беловолосый человек остановился у самого выхода из ложи. Пожилой че

In [29]:
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start=test_str_slov,temperature=i ,enc=enc)}\n")

--- Temperature = 0.3
Беловолосый человек остановился у самого по на го Нера по по пра сть берить ва верить Онали при нера по сто неро ма ста то при про сть во пра Я по по про сто сте на во несто сь сть слиму нерика прой сть вонерро ви прито мери сто о пра в но сть ста сть пра на сти нитов ка про неро про ста по понера по пра за ма о вера пра сть сть скоре пре

--- Temperature = 0.8
Беловолосый человек остановился у самогобы сиже, пала Га, и Норери чкото про повс — Га поте знна скильсесламит в ть за Вора знет ли, Я вере, сть мую, ей звся пре нелащи И пра онерошен содое вз Прелей лалаль ть Машетарри ныве ни — о лаза преза Гая, Лола Нарерио, сттди зожалла, веро — стириго см Вслакалутовра — мо нены клаза, ланы е че слит

--- Temperature = 1.0
Беловолосый человек остановился у самого о дарешавый вачено, нем ляце и Га Тымнучт, бозцы со — хос быкоятыви. паката — созалаючестьшьючтть Ода! фаросо, Га тня пруче проль Ная… мо, сть сло бунориче снитобыть иле ть. чка, зкл ― По ся пруль у. етуш-ть с

## Обучение Simple RNN с пословной токенизацией

In [30]:
nchars = 10
vocab_size = len(vocab_slov)

In [31]:
net=RNNGenerator(vocab_size,nchars).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [32]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch_slov, generate_slov, slov = True)

Epoch 0/10
Current loss T = 3.415301177496257
Current loss V = 3.086150640965509
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона
Epoch 1/10
Current loss T = 3.0527994057109185
Current loss V = 2.9130370901538836
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — 

In [33]:
print(f"Результат\n{generate_slov(net,size=300,start=test_str_slov)}\n")

Результат
Беловолосый человек остановился у самого начала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказала гермиона . — сказа